In [4]:
import tensorflow as tf 
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from scipy.stats import chi2_contingency 
from pipelines import preprocessor, preprocessor2
import pickle
import joblib

In [5]:
train= pd.read_csv("Datasets/train.csv")

<IPython.core.display.Javascript object>

In [58]:
X=train.iloc[:,:-1]
y=train.iloc[:, -1]

In [59]:
X.shape

(18506, 20)

In [60]:
X.head(2)

,Tour_ID,country,age_group,travel_with,total_female,total_male,purpose,main_activity,info_source,tour_arrangement,package_transport_int,package_accomodation,package_food,package_transport_tz,package_sightseeing,package_guided_tour,package_insurance,night_mainland,night_zanzibar,first_trip_tz
0,tour_id1hffseyw,ITALY,45-64,With Children,0.0,2.0,Visiting Friends and Relatives,Beach Tourism,"Friends, relatives",Package Tour,Yes,Yes,Yes,Yes,No,No,No,0,7,Yes
1,tour_idnacd7zag,UNITED KINGDOM,25-44,With Spouse,1.0,1.0,Leisure and Holidays,Wildlife Tourism,"Travel agent, tour operator",Package Tour,Yes,Yes,Yes,Yes,No,No,No,0,7,Yes


In [73]:
col_drop=[]

for col in X.columns:
    
    ct=pd.crosstab(y, X[col])
    chi2, P, _ , _ = chi2_contingency(ct)
    
    if P >= 0.05:
        col_drop.append(col)
    elif chi2 <= 3000:
            col_drop.append(col)
        
    print(f"Column: {col}\nChi_Test: {chi2}\nP_Value: {P}\n...............")

<IPython.core.display.Javascript object>

Column: Tour_ID
Chi_Test: 92530.0
P_Value: 0.4947449902995666
...............


<IPython.core.display.Javascript object>

Column: country
Chi_Test: 9779.033678192025
P_Value: 0.0
...............


<IPython.core.display.Javascript object>

...

Column: night_mainland
Chi_Test: 5614.015301227679
P_Value: 0.0
...............


<IPython.core.display.Javascript object>

Column: night_zanzibar
Chi_Test: 2652.1882265219597
P_Value: 0.0
...............


<IPython.core.display.Javascript object>

Column: first_trip_tz
Chi_Test: 2229.180451768042
P_Value: 0.0
...............


In [74]:
col_drop

['Tour_ID',
 'age_group',
 'total_male',
 'package_insurance',
 'night_zanzibar',
 'first_trip_tz']

In [75]:
X.drop(columns=col_drop, inplace=True)
X.shape

(18506, 14)

In [76]:
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()
y=le.fit_transform(y)
mapping= dict(zip(le.classes_, le.transform(le.classes_)))
print(mapping)

{'High Cost': 0, 'Higher Cost': 1, 'Highest Cost': 2, 'Low Cost': 3, 'Lower Cost': 4, 'Normal Cost': 5}


In [77]:
mapping

{'High Cost': 0,
 'Higher Cost': 1,
 'Highest Cost': 2,
 'Low Cost': 3,
 'Lower Cost': 4,
 'Normal Cost': 5}

In [80]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, shuffle=True, stratify= y, random_state=1)

print(f"X_train: {X_train.shape}\nX_test: {X_test.shape}\ny_train: {y_train.shape}\ny_test: {y_test.shape}")

X_train: (14804, 14)
X_test: (3702, 14)
y_train: (14804,)
y_test: (3702,)


In [ ]:
n_features= 

model= Sequential()